In [16]:
import pandas as pd
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ndcg_score, roc_auc_score
from typing import Tuple
import numpy as np

In [55]:
user_interest_df = pd.read_feather('../../datasets/raw_new/user_interest.feather')
user_df = pd.read_feather('../../datasets/raw_new/user.feather')
season_df = pd.read_feather('../../datasets/raw_new/season.feather')
video_df = pd.read_feather('../../datasets/raw_new/video.feather')
vote_df = pd.read_feather('../../datasets/raw_new/vote.feather')

In [10]:
user_interest_df = user_interest_df.groupby("user_id")["name"].apply(", ".join).reset_index(name="interests")

df = user_df["id"].to_frame().merge(user_interest_df, left_on="id", right_on="user_id", how="left", suffixes=["_user", "_interaction"])
df

,id,user_id,interests
0,00009b00-b3ae-458a-841e-e3ad5f1a2b3c,NaN,NaN
1,00009c76-592e-46a3-8754-59d68e1a2b3c,NaN,NaN
2,0000a373-24ff-4bb3-8ff1-f924161a2b3c,0000a373-24ff-4bb3-8ff1-f924161a2b3c,"ART&DESIGN, CRYPTO, FINANCE, GAMING, NFT"
3,00011b54-e070-4d0b-b8db-76cb9f1a2b3c,NaN,NaN
4,00013ce3-6800-4f34-914b-6c99171a2b3c,NaN,NaN
...,...,...,...
67783,fffc61cb-db48-450d-a721-30fc8a1a2b3c,NaN,NaN
67784,fffcc67a-b6b3-4246-a554-7efc2f1a2b3c,NaN,NaN
67785,fffce31b-6034-4d11-8593-a576061a2b3c,NaN,NaN
67786,fffeb5ee-8285-4a08-806d-d07ff91a2b3c,NaN,NaN


In [11]:
upvotes_df = vote_df.groupby(["voter_id", "video_id"])["id"].nunique().reset_index(name="upvotes")
upvotes_df

,voter_id,video_id,upvotes
0,000064da-cce1-4dec-929f-ea2d871a2b3c,fd4e8fd6-9524-4790-b325-7170411a2b3c,1
1,000068ac-49ae-421a-86ee-83ea321a2b3c,ebe8ccd7-caa4-40d5-bcbd-1d21c11a2b3c,1
2,00009c76-592e-46a3-8754-59d68e1a2b3c,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,12
3,00009c76-592e-46a3-8754-59d68e1a2b3c,c7716a3c-5a03-4563-8364-823ebd1a2b3c,1
4,000233b9-d92e-4c07-9bf7-f5d8911a2b3c,fd4e8fd6-9524-4790-b325-7170411a2b3c,1
...,...,...,...
142299,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,67dad23b-08c9-4097-98ac-dfd13d1a2b3c,1
142300,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,8ae653d5-ba1c-4a17-92ce-f973681a2b3c,2
142301,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,95c9270c-e6db-4520-823a-447aee1a2b3c,1
142302,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,c05f4ba5-8830-4be5-808c-8706e81a2b3c,2


In [13]:
df = df.merge(upvotes_df, left_on="id", right_on="voter_id", how="left")
df = df[df["video_id"].notna()]
df

,id,user_id,interests,voter_id,video_id,upvotes
1,00009c76-592e-46a3-8754-59d68e1a2b3c,NaN,NaN,00009c76-592e-46a3-8754-59d68e1a2b3c,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,12.0
2,00009c76-592e-46a3-8754-59d68e1a2b3c,NaN,NaN,00009c76-592e-46a3-8754-59d68e1a2b3c,c7716a3c-5a03-4563-8364-823ebd1a2b3c,1.0
6,000233b9-d92e-4c07-9bf7-f5d8911a2b3c,NaN,NaN,000233b9-d92e-4c07-9bf7-f5d8911a2b3c,fd4e8fd6-9524-4790-b325-7170411a2b3c,1.0
8,0002a16e-70bc-4fb9-9fd8-2199841a2b3c,NaN,NaN,0002a16e-70bc-4fb9-9fd8-2199841a2b3c,8442dd86-17b2-4887-983d-bc62c21a2b3c,1.0
9,0002bcf0-3a0e-4850-81a9-7961d61a2b3c,NaN,NaN,0002bcf0-3a0e-4850-81a9-7961d61a2b3c,ebe8ccd7-caa4-40d5-bcbd-1d21c11a2b3c,1.0
...,...,...,...,...,...,...
130814,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,67dad23b-08c9-4097-98ac-dfd13d1a2b3c,1.0
130815,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,8ae653d5-ba1c-4a17-92ce-f973681a2b3c,2.0
130816,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,95c9270c-e6db-4520-823a-447aee1a2b3c,1.0
130817,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,c05f4ba5-8830-4be5-808c-8706e81a2b3c,2.0


In [14]:
df = df.merge(video_df[["id", "season_id", "created_at"]], left_on="video_id", right_on="id", how="left", suffixes=["_user", "_video"])
df

,id_user,user_id,interests,voter_id,video_id,upvotes,id_video,season_id,created_at
0,00009c76-592e-46a3-8754-59d68e1a2b3c,NaN,NaN,00009c76-592e-46a3-8754-59d68e1a2b3c,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,12.0,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,43fda735-831b-41e7-9100-39657e1a2b3c,2023-06-14 17:18:31
1,00009c76-592e-46a3-8754-59d68e1a2b3c,NaN,NaN,00009c76-592e-46a3-8754-59d68e1a2b3c,c7716a3c-5a03-4563-8364-823ebd1a2b3c,1.0,c7716a3c-5a03-4563-8364-823ebd1a2b3c,43fda735-831b-41e7-9100-39657e1a2b3c,2023-06-15 06:24:14
2,000233b9-d92e-4c07-9bf7-f5d8911a2b3c,NaN,NaN,000233b9-d92e-4c07-9bf7-f5d8911a2b3c,fd4e8fd6-9524-4790-b325-7170411a2b3c,1.0,fd4e8fd6-9524-4790-b325-7170411a2b3c,46b1f1e7-b2ab-4e16-a275-fb63081a2b3c,2023-04-22 02:04:19
3,0002a16e-70bc-4fb9-9fd8-2199841a2b3c,NaN,NaN,0002a16e-70bc-4fb9-9fd8-2199841a2b3c,8442dd86-17b2-4887-983d-bc62c21a2b3c,1.0,8442dd86-17b2-4887-983d-bc62c21a2b3c,46b1f1e7-b2ab-4e16-a275-fb63081a2b3c,2023-04-22 02:04:19
4,0002bcf0-3a0e-4850-81a9-7961d61a2b3c,NaN,NaN,0002bcf0-3a0e-4850-81a9-7961d61a2b3c,ebe8ccd7-caa4-40d5-bcbd-1d21c11a2b3c,1.0,ebe8ccd7-caa4-40d5-bcbd-1d21c11a2b3c,046a6b76-bb19-4619-bbf5-2ad9e71a2b3c,2023-04-22 02:04:19
...,...,...,...,...,...,...,...,...,...
107525,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,67dad23b-08c9-4097-98ac-dfd13d1a2b3c,1.0,67dad23b-08c9-4097-98ac-dfd13d1a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,2023-05-02 03:38:07
107526,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,8ae653d5-ba1c-4a17-92ce-f973681a2b3c,2.0,8ae653d5-ba1c-4a17-92ce-f973681a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,2023-05-02 02:54:38
107527,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,95c9270c-e6db-4520-823a-447aee1a2b3c,1.0,95c9270c-e6db-4520-823a-447aee1a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,2023-05-02 03:36:59
107528,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,c05f4ba5-8830-4be5-808c-8706e81a2b3c,2.0,c05f4ba5-8830-4be5-808c-8706e81a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,2023-05-02 03:31:12


In [15]:
df = df.merge(season_df[["id", "category"]], left_on="season_id", right_on="id", how="left")
df

,id_user,user_id,interests,voter_id,video_id,upvotes,id_video,season_id,created_at,id,category
0,00009c76-592e-46a3-8754-59d68e1a2b3c,NaN,NaN,00009c76-592e-46a3-8754-59d68e1a2b3c,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,12.0,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,43fda735-831b-41e7-9100-39657e1a2b3c,2023-06-14 17:18:31,43fda735-831b-41e7-9100-39657e1a2b3c,OTHERS
1,00009c76-592e-46a3-8754-59d68e1a2b3c,NaN,NaN,00009c76-592e-46a3-8754-59d68e1a2b3c,c7716a3c-5a03-4563-8364-823ebd1a2b3c,1.0,c7716a3c-5a03-4563-8364-823ebd1a2b3c,43fda735-831b-41e7-9100-39657e1a2b3c,2023-06-15 06:24:14,43fda735-831b-41e7-9100-39657e1a2b3c,OTHERS
2,000233b9-d92e-4c07-9bf7-f5d8911a2b3c,NaN,NaN,000233b9-d92e-4c07-9bf7-f5d8911a2b3c,fd4e8fd6-9524-4790-b325-7170411a2b3c,1.0,fd4e8fd6-9524-4790-b325-7170411a2b3c,46b1f1e7-b2ab-4e16-a275-fb63081a2b3c,2023-04-22 02:04:19,46b1f1e7-b2ab-4e16-a275-fb63081a2b3c,DANCE
3,0002a16e-70bc-4fb9-9fd8-2199841a2b3c,NaN,NaN,0002a16e-70bc-4fb9-9fd8-2199841a2b3c,8442dd86-17b2-4887-983d-bc62c21a2b3c,1.0,8442dd86-17b2-4887-983d-bc62c21a2b3c,46b1f1e7-b2ab-4e16-a275-fb63081a2b3c,2023-04-22 02:04:19,46b1f1e7-b2ab-4e16-a275-fb63081a2b3c,DANCE
4,0002bcf0-3a0e-4850-81a9-7961d61a2b3c,NaN,NaN,0002bcf0-3a0e-4850-81a9-7961d61a2b3c,ebe8ccd7-caa4-40d5-bcbd-1d21c11a2b3c,1.0,ebe8ccd7-caa4-40d5-bcbd-1d21c11a2b3c,046a6b76-bb19-4619-bbf5-2ad9e71a2b3c,2023-04-22 02:04:19,046a6b76-bb19-4619-bbf5-2ad9e71a2b3c,DANCE
...,...,...,...,...,...,...,...,...,...,...,...
107525,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,67dad23b-08c9-4097-98ac-dfd13d1a2b3c,1.0,67dad23b-08c9-4097-98ac-dfd13d1a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,2023-05-02 03:38:07,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,STYLE&BEAUTY
107526,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,8ae653d5-ba1c-4a17-92ce-f973681a2b3c,2.0,8ae653d5-ba1c-4a17-92ce-f973681a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,2023-05-02 02:54:38,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,STYLE&BEAUTY
107527,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,95c9270c-e6db-4520-823a-447aee1a2b3c,1.0,95c9270c-e6db-4520-823a-447aee1a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,2023-05-02 03:36:59,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,STYLE&BEAUTY
107528,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,NaN,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,c05f4ba5-8830-4be5-808c-8706e81a2b3c,2.0,c05f4ba5-8830-4be5-808c-8706e81a2b3c,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,2023-05-02 03:31:12,9f6786f4-fc20-45c2-a8c5-944e941a2b3c,STYLE&BEAUTY


In [16]:
end_date = df["created_at"].max() - timedelta(weeks=2)
train = df[df["created_at"] < end_date][["id_user", "interests", "video_id", "category", "upvotes"]] 
train

,id_user,interests,video_id,category,upvotes
0,00009c76-592e-46a3-8754-59d68e1a2b3c,NaN,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,OTHERS,12.0
1,00009c76-592e-46a3-8754-59d68e1a2b3c,NaN,c7716a3c-5a03-4563-8364-823ebd1a2b3c,OTHERS,1.0
2,000233b9-d92e-4c07-9bf7-f5d8911a2b3c,NaN,fd4e8fd6-9524-4790-b325-7170411a2b3c,DANCE,1.0
3,0002a16e-70bc-4fb9-9fd8-2199841a2b3c,NaN,8442dd86-17b2-4887-983d-bc62c21a2b3c,DANCE,1.0
4,0002bcf0-3a0e-4850-81a9-7961d61a2b3c,NaN,ebe8ccd7-caa4-40d5-bcbd-1d21c11a2b3c,DANCE,1.0
...,...,...,...,...,...
107525,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,67dad23b-08c9-4097-98ac-dfd13d1a2b3c,STYLE&BEAUTY,1.0
107526,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,8ae653d5-ba1c-4a17-92ce-f973681a2b3c,STYLE&BEAUTY,2.0
107527,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,95c9270c-e6db-4520-823a-447aee1a2b3c,STYLE&BEAUTY,1.0
107528,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,NaN,c05f4ba5-8830-4be5-808c-8706e81a2b3c,STYLE&BEAUTY,2.0


In [17]:
enc = OrdinalEncoder(encoded_missing_value=-1)
enc.fit(df[["id_user", "interests", "video_id", "category"]])
train[["id_user", "interests", "video_id", "category"]] = enc.transform(train[["id_user", "interests", "video_id", "category"]])
train

,id_user,interests,video_id,category,upvotes
0,0.0,-1.0,0.0,9.0,12.0
1,0.0,-1.0,2043.0,9.0,1.0
2,1.0,-1.0,2580.0,2.0,1.0
3,2.0,-1.0,1354.0,2.0,1.0
4,3.0,-1.0,2423.0,2.0,1.0
...,...,...,...,...,...
107525,44498.0,-1.0,1072.0,11.0,1.0
107526,44498.0,-1.0,1421.0,11.0,2.0
107527,44498.0,-1.0,1547.0,11.0,1.0
107528,44498.0,-1.0,1961.0,11.0,2.0


In [18]:
model = RandomForestClassifier()
model.fit(train.loc[:, train.columns != "upvotes"], train["upvotes"])

RandomForestClassifier()

In [19]:
test_upvotes = df[df["created_at"] >= end_date][["id_user", "interests", "video_id", "category", "upvotes"]] 
test = df[["id_user", "interests"]].drop_duplicates("id_user").merge(df[["video_id", "category"]].drop_duplicates("category"), how="cross")
test = pd.DataFrame(enc.transform(test), columns=test.columns)
test = pd.merge(test, train[["id_user", "interests", "video_id", "category"]], indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
test

,id_user,interests,video_id,category
1,0.0,-1.0,2580.0,2.0
2,0.0,-1.0,1944.0,10.0
3,0.0,-1.0,1419.0,11.0
4,0.0,-1.0,475.0,-1.0
5,0.0,-1.0,1771.0,6.0
...,...,...,...,...
578482,44498.0,-1.0,391.0,1.0
578483,44498.0,-1.0,2081.0,8.0
578484,44498.0,-1.0,7.0,4.0
578485,44498.0,-1.0,37.0,0.0


In [20]:
test["prediction"] = model.predict(test)
test

,id_user,interests,video_id,category,prediction
1,0.0,-1.0,2580.0,2.0,1.0
2,0.0,-1.0,1944.0,10.0,1.0
3,0.0,-1.0,1419.0,11.0,1.0
4,0.0,-1.0,475.0,-1.0,1.0
5,0.0,-1.0,1771.0,6.0,1.0
...,...,...,...,...,...
578482,44498.0,-1.0,391.0,1.0,2.0
578483,44498.0,-1.0,2081.0,8.0,1.0
578484,44498.0,-1.0,7.0,4.0,2.0
578485,44498.0,-1.0,37.0,0.0,1.0


In [21]:
test.sort_values("prediction", ascending=False)

,id_user,interests,video_id,category,prediction
376247,28942.0,-1.0,2580.0,2.0,18.0
376325,28948.0,-1.0,2580.0,2.0,18.0
376312,28947.0,-1.0,2580.0,2.0,18.0
376221,28940.0,-1.0,2580.0,2.0,18.0
376273,28944.0,-1.0,2580.0,2.0,18.0
...,...,...,...,...,...
212326,16332.0,-1.0,7.0,4.0,1.0
212324,16332.0,-1.0,391.0,1.0,1.0
212323,16332.0,-1.0,2532.0,3.0,1.0
212321,16332.0,-1.0,1771.0,6.0,1.0


In [24]:
test[test["prediction"] == 18]["video_id"].nunique()

1

In [18]:
def get_end_date() -> str:
    # Calculate end date as 2 weeks before today
    today = datetime.now()
    end_date = (today - timedelta(weeks=2)).strftime('%Y-%m-%d')
    return end_date

def get_num_cycles(start_date: str) -> int:
    # Get today's date
    today_date = datetime.now().strftime('%Y-%m-%d')
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
    end_datetime = datetime.strptime(today_date, '%Y-%m-%d')
    date_difference = (end_datetime - start_datetime).days

    return date_difference

def train_test_split_for_data(data, date_col, start_date):
    train_data = data[data[date_col] <= start_date]
    test_data = data[(data[date_col] > start_date) & (data[date_col] <= get_end_date())]
    return train_data, test_data

def create_interaction_df(user_interest_df: pd.DataFrame, user_df: pd.DataFrame, season_df: pd.DataFrame, 
                              video_df: pd.DataFrame, vote_df: pd.DataFrame, date: datetime):
    user_interest_df = user_df["id"].to_frame().merge(user_interest_df, left_on="id", right_on="user_id", how="left", suffixes=["_user", "_interaction"])
    user_interest_df["count"] = 1
    user_interest_df = user_interest_df.pivot(index="id", columns="name", values="count")
    user_interest_df = user_interest_df.loc[:, user_interest_df.columns.notna()].fillna(0)

    upvotes_df = vote_df[(vote_df["created_at"] > date - timedelta(weeks=1)) & (vote_df["created_at"] <= date)].groupby(["voter_id", "video_id"])["id"].nunique().reset_index(name="upvotes")
    upvotes_df = upvotes_df.merge(video_df[["id", "season_id", "created_at"]], left_on="video_id", right_on="id", how="left", suffixes=["", "_video"])

    upvotes_category_df = upvotes_df.merge(season_df[["id", "category"]], left_on="season_id", right_on="id", how="left", suffixes=["_video", "_season"])
    upvotes_category_df = upvotes_category_df.pivot_table(columns="category", index="voter_id", values="upvotes", aggfunc=sum, fill_value=0)
    upvotes_category_df["CRYPTO"] = 0
    upvotes_category_df["FINANCE"] = 0

    user_interest_df = user_interest_df.add(upvotes_category_df, fill_value=0)
    user_interest_df.reset_index(names="user_id", inplace=True)
    user_interest_df = user_interest_df[user_interest_df.loc[:, user_interest_df.columns != "user_id"].sum(1) > 0]

    interaction_df = user_interest_df.merge(upvotes_df[["voter_id", "video_id", "upvotes"]], left_on="user_id", right_on="voter_id", how="left", suffixes=["_user", "_upvotes"])
    interaction_df.drop(columns="voter_id", inplace=True)
    interaction_df.dropna(subset="video_id", axis=0, inplace=True)
    
    video_category_df = video_df[["id", "created_at", "season_id"]].merge(season_df[["id", "category"]], left_on="season_id", right_on="id", how="left", suffixes=["_video", "_season"])

    interaction_df = interaction_df.merge(video_category_df, left_on="video_id", right_on="id_video", how="left")
    interaction_df.drop(columns=["id_video", "season_id", "id_season"], inplace=True)
    interaction_df.set_index(["user_id", "video_id"], inplace=True)
    
    test_df = user_interest_df.merge(video_category_df[(video_category_df["created_at"] > date) & (video_category_df["created_at"] <= get_end_date())], how="cross")
    test_df.drop(columns=["season_id", "id_season"], inplace=True)
    test_df.rename(columns={"id_video": "video_id"}, inplace=True)
    test_df.set_index(["user_id", "video_id"], inplace=True)

    enc = LabelEncoder(encoded_missing_value=-1)
    enc.fit(video_category_df["category"].to_frame())
    interaction_df["category"] = enc.transform(interaction_df["category"].to_frame())
    test_df["category"] = enc.transform(test_df["category"].to_frame())
    interaction_df.fillna(0, inplace=True)
    test_df.fillna(0, inplace=True)
    
    return interaction_df, test_df

def find_top_k_videos(user_id, k, prediction_df):
    return prediction_df[prediction_df.index.get_level_values("user_id") == user_id].nlargest(k, "prediction")

def hit_ratio_at_k(y_true, y_pred, K):
    top_k_indices = np.argsort(-np.array(y_pred))[:K]
    return int(any(y_true[i] == 1 for i in top_k_indices))  # 1 if at least one relevant item is in top-K, 0 otherwise

def ndcg_at_k(y_true, y_pred, K):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred) 
    return ndcg_score([y_true], [y_pred], k=K)

def get_summary_statistics(vote_df, train_df, test_df, date, K):
    train_df.drop(columns="created_at", inplace=True)
    test_df.drop(columns="created_at", inplace=True)
    vote_test = vote_df[vote_df["created_at"] > date]
    vote_test['created_at'] = vote_test['created_at'].dt.date
    model_statistics = pd.DataFrame(columns=['dt', 'roc_auc_score', 'accuracy', 'precision', 'recall', 'f1_score', 'hit_ratio_k', 'ndcg_k'])
    model = RandomForestClassifier()
    model.fit(train_df.loc[:, train_df.columns != "upvotes"], train_df["upvotes"])
    test_df["prediction"] = model.predict(test_df)

    for day in sorted(vote_test['created_at'].unique()):
        print(day)
        voted_videos_for_day = vote_test[vote_test['created_at'] == day]
        summary_statistics = pd.DataFrame(columns=['user_id', 'roc_auc_score', 'accuracy', 'precision', 'recall', 'f1_score', 'hit_ratio_k', 'ndcg_k'])

        for user_id in voted_videos_for_day['voter_id'].unique():
            user_voted_videos = voted_videos_for_day[voted_videos_for_day['voter_id'] == user_id]
            
            if user_id not in train_df.index or len(set(user_voted_videos["video_id"]) & set(test_df.index.get_level_values("video_id"))) == 0:
                continue
            
#             create dataframe to calculate confusion matrix
            y_true_and_pred = pd.DataFrame(index=test_df.index.get_level_values("video_id"))
            y_true_and_pred['true'] = np.where(y_true_and_pred.index.isin(user_voted_videos['video_id']), 1, 0)

            recommendations = find_top_k_videos(user_id, 20, test_df)
            y_true_and_pred['pred'] = np.where(y_true_and_pred.index.isin(recommendations.index), 1, 0)

            try:
                roc_auc = roc_auc_score(y_true_and_pred['true'], y_true_and_pred['pred'])
                accuracy = accuracy_score(y_true_and_pred['true'], y_true_and_pred['pred'])
                precision = precision_score(y_true_and_pred['true'], y_true_and_pred['pred'])
                recall = recall_score(y_true_and_pred['true'], y_true_and_pred['pred'])
                f1 = f1_score(y_true_and_pred['true'], y_true_and_pred['pred'])
                hit_ratio = hit_ratio_at_k(y_true_and_pred['true'], y_true_and_pred['pred'], K)
                ndcg = ndcg_at_k(y_true_and_pred['true'], y_true_and_pred['pred'], K)
            except ValueError:
                print(f'ROC AUC for {user_id} not valid')

            summary_statistics.loc[len(summary_statistics)] = [user_id, roc_auc, accuracy, precision, recall, f1, hit_ratio, ndcg]
        
        model_statistics.loc[len(model_statistics)] = np.append(np.array(day), summary_statistics.iloc[:,1:].mean().values)

    return model_statistics

def run_random_forest(date, K, num_cycles):
    user_interest_df = pd.read_feather('../../datasets/raw_new/user_interest.feather')
    user_df = pd.read_feather('../../datasets/raw_new/user.feather')
    season_df = pd.read_feather('../../datasets/raw_new/season.feather')
    video_df = pd.read_feather('../../datasets/raw_new/video.feather')
    vote_df = pd.read_feather('../../datasets/raw_new/vote.feather')

    date = pd.to_datetime(date)

    model_statistics = pd.DataFrame(columns=['dt', 'roc_auc_score', 'accuracy', 'precision', 'recall', 'f1_score', 'hit_ratio_k', 'ndcg_k'])
    # for cycle in range(num_cycles):
    train_df, test_df = create_interaction_df(user_interest_df, user_df, season_df, video_df, vote_df, date)
    model_statistics_for_training_cycle = get_summary_statistics(vote_df, train_df, test_df, date, K)
    model_statistics = pd.concat([model_statistics, model_statistics_for_training_cycle])
    # date = get_end_date()

    model_statistics['model'] = 'random_forest'
    model_statistics.to_csv('datasets/final/random_forest_video.csv', index=False)

In [83]:
random_forest_eval_video = run_random_forest('2023-07-01', 10, get_num_cycles('2023-07-01'))

/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_83969/1496672927.py:32: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  upvotes_category_df = upvotes_category_df.pivot_table(columns="category", index="voter_id", values="upvotes", aggfunc=sum, fill_value=0)
/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_83969/1496672927.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  interaction_df.fillna(0, inplace=True)
/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_83969/1496672927.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

2023-07-01


/Users/lozhaowei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_83969/1496672927.py:69: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return int(any(y_true[i] == 1 for i in top_k_indices))  # 1 if at least one relevant item is in top-K, 0 otherwise
/Users/lozhaowei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _

/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_83969/1496672927.py:69: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return int(any(y_true[i] == 1 for i in top_k_indices))  # 1 if at least one relevant item is in top-K, 0 otherwise
/Users/lozhaowei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_83969/1496672927.py:69: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behav

2023-07-02


...

/Users/lozhaowei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_83969/1496672927.py:69: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return int(any(y_true[i] == 1 for i in top_k_indices))  # 1 if at least one relevant item is in top-K, 0 otherwise
/Users/lozhaowei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _

/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_83969/1496672927.py:69: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return int(any(y_true[i] == 1 for i in top_k_indices))  # 1 if at least one relevant item is in top-K, 0 otherwise
/Users/lozhaowei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_83969/1496672927.py:69: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behav

2023-09-06


/Users/lozhaowei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_83969/1496672927.py:69: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return int(any(y_true[i] == 1 for i in top_k_indices))  # 1 if at least one relevant item is in top-K, 0 otherwise
/Users/lozhaowei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _

OSError: Cannot save file into a non-existent directory: 'datasets/final'

In [19]:
train_df, test_df = create_interaction_df(user_interest_df, user_df, season_df, video_df, vote_df, pd.to_datetime("2023-07-01"))

/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_38667/720440519.py:32: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  upvotes_category_df = upvotes_category_df.pivot_table(columns="category", index="voter_id", values="upvotes", aggfunc=sum, fill_value=0)


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [10]:
# train_df.drop(columns="created_at", inplace=True)
# test_df.drop(columns="created_at", inplace=True)
vote_test = vote_df[vote_df["created_at"] > "2023-07-01"]
vote_test['created_at'] = vote_test['created_at'].dt.date
model_statistics = pd.DataFrame(columns=['dt', 'roc_auc_score', 'accuracy', 'precision', 'recall', 'f1_score', 'hit_ratio_k', 'ndcg_k'])
model = RandomForestClassifier()
model.fit(train_df.loc[:, train_df.columns != "upvotes"], train_df["upvotes"])
test_df["prediction"] = model.predict(test_df)

/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_38667/1196438857.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vote_test['created_at'] = vote_test['created_at'].dt.date


In [15]:
voted_videos_for_day = vote_test[vote_test['created_at'] == "2023-07-02"]
y_true_and_pred = pd.DataFrame(index=test_df.index.get_level_values("video_id"))
user_voted_videos = voted_videos_for_day[voted_videos_for_day['voter_id'] == voted_videos_for_day["voter_id"].iloc[0]]
            
if user_id not in train_df.index or len(set(user_voted_videos["video_id"]) & set(test_df.index.get_level_values("video_id"))) == 0:
    continue

#             create dataframe to calculate confusion matrix
y_true_and_pred = pd.DataFrame(index=test_df.index.get_level_values("video_id"))
y_true_and_pred['true'] = np.where(y_true_and_pred.index.isin(user_voted_videos['video_id']), 1, 0)

recommendations = find_top_k_videos(user_id, 20, test_df)
y_true_and_pred['pred'] = np.where(y_true_and_pred.index.isin(recommendations.index), 1, 0)


        


IndexError: single positional indexer is out-of-bounds

In [56]:
date = pd.to_datetime("2023-08-01")
user_interest_df = user_df["id"].to_frame().merge(user_interest_df, left_on="id", right_on="user_id", how="left", suffixes=["_user", "_interaction"])
user_interest_df["count"] = 1
user_interest_df = user_interest_df.pivot(index="id", columns="name", values="count")
user_interest_df = user_interest_df.loc[:, user_interest_df.columns.notna()].fillna(0)

upvotes_df = vote_df[vote_df["created_at"] <= date].groupby(["voter_id", "video_id"])["id"].nunique().reset_index(name="upvotes")
upvotes_df = upvotes_df.merge(video_df[["id", "season_id", "created_at"]], left_on="video_id", right_on="id", how="left", suffixes=["", "_video"])

upvotes_category_df = upvotes_df.merge(season_df[["id", "category"]], left_on="season_id", right_on="id", how="left", suffixes=["_video", "_season"])
upvotes_category_df = upvotes_category_df.pivot_table(columns="category", index="voter_id", values="upvotes", aggfunc=sum, fill_value=0)
upvotes_category_df["CRYPTO"] = 0
upvotes_category_df["FINANCE"] = 0

user_interest_df = user_interest_df.add(upvotes_category_df, fill_value=0)
user_interest_df.reset_index(names="user_id", inplace=True)
user_interest_df = user_interest_df[user_interest_df.loc[:, user_interest_df.columns != "user_id"].sum(1) > 0]

interaction_df = user_interest_df.merge(upvotes_df[["voter_id", "video_id", "upvotes"]], left_on="user_id", right_on="voter_id", how="left", suffixes=["_user", "_upvotes"])
interaction_df.drop(columns="voter_id", inplace=True)
interaction_df.dropna(subset="video_id", axis=0, inplace=True)

video_category_df = video_df[["id", "created_at", "season_id"]].merge(season_df[["id", "category"]], left_on="season_id", right_on="id", how="left", suffixes=["_video", "_season"])

interaction_df = interaction_df.merge(video_category_df, left_on="video_id", right_on="id_video", how="left")
interaction_df.drop(columns=["id_video", "season_id", "id_season"], inplace=True)
interaction_df.set_index(["user_id", "video_id"], inplace=True)

test_df = user_interest_df.merge(video_category_df[(video_category_df["created_at"] > date) & (video_category_df["created_at"] <= get_end_date())], how="cross")
test_df.drop(columns=["season_id", "id_season"], inplace=True)
test_df.rename(columns={"id_video": "video_id"}, inplace=True)
test_df.set_index(["user_id", "video_id"], inplace=True)

/var/folders/vh/63y39_z94rl49c3q3lgz05vr0000gn/T/ipykernel_38667/3186917450.py:11: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  upvotes_category_df = upvotes_category_df.pivot_table(columns="category", index="voter_id", values="upvotes", aggfunc=sum, fill_value=0)


In [43]:
upvotes_df = vote_df[(vote_df["created_at"] > date - timedelta(weeks=1)) & (vote_df["created_at"] <= date)].groupby(["voter_id", "video_id"])["id"].nunique().reset_index(name="upvotes")

In [45]:
upvotes_df = upvotes_df.merge(video_df[["id", "season_id", "created_at"]], left_on="video_id", right_on="id", how="left", suffixes=["", "_video"])

In [57]:
upvotes_df

,voter_id,video_id,upvotes,id,season_id,created_at
0,000064da-cce1-4dec-929f-ea2d871a2b3c,fd4e8fd6-9524-4790-b325-7170411a2b3c,1,NaN,NaN,NaT
1,000068ac-49ae-421a-86ee-83ea321a2b3c,ebe8ccd7-caa4-40d5-bcbd-1d21c11a2b3c,1,NaN,NaN,NaT
2,00009c76-592e-46a3-8754-59d68e1a2b3c,00033275-cbd8-4ee5-8c1f-7a50491a2b3c,12,NaN,NaN,NaT
3,00009c76-592e-46a3-8754-59d68e1a2b3c,c7716a3c-5a03-4563-8364-823ebd1a2b3c,1,NaN,NaN,NaT
4,000233b9-d92e-4c07-9bf7-f5d8911a2b3c,fd4e8fd6-9524-4790-b325-7170411a2b3c,1,NaN,NaN,NaT
...,...,...,...,...,...,...
136879,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,67dad23b-08c9-4097-98ac-dfd13d1a2b3c,1,NaN,NaN,NaT
136880,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,8ae653d5-ba1c-4a17-92ce-f973681a2b3c,2,NaN,NaN,NaT
136881,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,95c9270c-e6db-4520-823a-447aee1a2b3c,1,NaN,NaN,NaT
136882,ffff7f3c-c253-45e0-a881-e7988c1a2b3c,c05f4ba5-8830-4be5-808c-8706e81a2b3c,2,NaN,NaN,NaT


In [48]:
video_df

,id,creator_id,season_id,created_at,recce_status
0,0000547f-39be-4da8-bd33-4e3dc6b843c3,b7ead459-9d8f-4025-8cff-28bd5b477ed5,2f02dfba-468b-4475-8922-9f3d47b42f8d,2023-09-21 21:15:46,ACTIVE
1,00033275-cbd8-4ee5-8c1f-7a5049862759,8f849ec4-b434-49d9-9b0d-c3e517ec9e31,43fda735-831b-41e7-9100-39657e0fc0d5,2023-06-14 17:18:31,ACTIVE
2,0033ee4a-49ad-4ec5-81ab-79f54944f018,4efc5762-78e6-4026-9e57-2ab5da5b629f,26079bd3-b493-4e6b-a929-dfb7f3052d70,2023-04-29 12:18:50,DELETED
3,00372015-eb6c-4e03-acab-07cf301bd8d0,20436a91-86ff-4b61-9003-6faccb803971,d375ece1-51fe-4ee5-850f-fb5c2df5b7e9,2023-09-17 20:12:27,ACTIVE
4,0038b147-833e-4843-a91e-c42bc54f3110,67ef2b79-d8ce-4345-a53e-b70072151421,34489240-0b1a-4567-82d9-f132249694a5,2023-09-25 06:11:00,ACTIVE
...,...,...,...,...,...
10630,ffe8e822-6fb3-4c7d-8b3a-7556f4aba35d,2f98d0ca-26f2-4bbc-9091-b582397789c0,43fda735-831b-41e7-9100-39657e0fc0d5,2023-09-17 09:03:38,ACTIVE
10631,ffe9936a-1f96-41ae-a929-357000bdd0e9,1c57e292-a19f-415a-8d9d-f47e404fe1fe,c36d1072-20a0-4c76-9bf4-3b610e4eb424,2023-09-16 10:51:12,ACTIVE
10632,ffea92d5-79c0-46bc-acf5-37d2f6dc944a,fc529c8f-f641-4b6a-b56f-0d359e3c5adf,e8795b31-de60-4fbc-8f71-1971d850f67e,2023-04-22 02:04:19,ACTIVE
10633,fff23e54-e714-4aed-8137-950f229a6177,695f13bd-0766-475d-86ad-ad01c379e350,d4aea959-24a5-4bb8-8b1d-606854872d77,2023-09-26 16:29:08,ACTIVE
